# 🚦 Memory-Safe Feature Extraction Pipeline (Restart Kernel Between Models!)

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/zara-clothes-image-data/metadata.csv
/kaggle/input/zara-clothes-image-data/images/3393757_2.jpg
/kaggle/input/zara-clothes-image-data/images/3339074_2.jpg
/kaggle/input/zara-clothes-image-data/images/3432147_1.jpg
/kaggle/input/zara-clothes-image-data/images/3458052_2.jpg
/kaggle/input/zara-clothes-image-data/images/3892302_1.jpg
/kaggle/input/zara-clothes-image-data/images/3453378_1.jpg
/kaggle/input/zara-clothes-image-data/images/3431525_2.jpg
/kaggle/input/zara-clothes-image-data/images/3432862_2.jpg
/kaggle/input/zara-clothes-image-data/images/3305271_2.jpg
/kaggle/input/zara-clothes-image-data/images/3347929_1.jpg
/kaggle/input/zara-clothes-image-data/images/3395849_1.jpg
/kaggle/input/zara-clothes-image-data/images/3425076_2.jpg
/kaggle/input/zara-clothes-image-data/images/3409961_1.jpg
/kaggle/input/zara-clothes-image-data/images/3229777_1.jpg
/kaggle/input/zara-clothes-image-data/images/3665060_1.jpg
/kaggle/input/zara-clothes-image-data/images/3240132_2.jpg
/kagg

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.6 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import faiss
import numpy as np
from PIL import Image
import tensorflow as tf
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from transformers import ViTImageProcessor, ViTModel
import torch
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

2025-06-22 19:33:48.625205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750620828.806305      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750620828.854232      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 0️⃣ Preparation: Data Paths and Image Lists (Run once after each restart)

In [8]:
import os
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image

# For Keras models (EfficientNet, ConvNeXt)
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# For FAISS
import faiss

# Paths (change these to your dataset locations!)
MAIN_DATASET_DIR = '/kaggle/input/clothes-dataset/Clothes_Dataset/'
ZARA_DIR = '/kaggle/input/zara-clothes-image-data/images/'

# Collect all image paths for main dataset
img_paths = []
labels = []
for label in sorted(os.listdir(MAIN_DATASET_DIR)):
    class_dir = os.path.join(MAIN_DATASET_DIR, label)
    if os.path.isdir(class_dir):
        for fname in os.listdir(class_dir):
            if fname.lower().endswith(('jpg', 'jpeg', 'png')):
                img_paths.append(os.path.join(class_dir, fname))
                labels.append(label)

# Collect Zara query image paths
zara_img_paths = [os.path.join(ZARA_DIR, fname) for fname in os.listdir(ZARA_DIR) if fname.lower().endswith(('jpg', 'jpeg', 'png'))]

print(f"Main dataset images: {len(img_paths)}")
print(f"Zara query images: {len(zara_img_paths)}")

Main dataset images: 7500
Zara query images: 834


## 1️⃣ EfficientNetV2 Feature Extraction

In [8]:
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as efficientnet_preprocess

efficientnet_model = EfficientNetV2L(weights="imagenet", include_top=False, pooling="avg")

def extract_features_batch_efficientnet(img_paths):
    batch_images = []
    for img_path in img_paths:
        img = image.load_img(img_path, target_size=(384, 384))
        x = image.img_to_array(img)
        batch_images.append(x)
    batch_images = np.array(batch_images)
    batch_images = efficientnet_preprocess(batch_images)
    feats = efficientnet_model.predict(batch_images, batch_size=len(batch_images), verbose=0)
    return feats

# Main dataset
features_efficientnet = []
BATCH_SIZE = 32
for start in tqdm(range(0, len(img_paths), BATCH_SIZE)):
    batch = img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_efficientnet(batch)
    features_efficientnet.extend(feats)
features_efficientnet = np.array(features_efficientnet).astype('float32')
np.save("features_efficientnet.npy", features_efficientnet)

# Zara queries
zara_features_efficientnet = []
for start in tqdm(range(0, len(zara_img_paths), BATCH_SIZE)):
    batch = zara_img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_efficientnet(batch)
    zara_features_efficientnet.extend(feats)
zara_features_efficientnet = np.array(zara_features_efficientnet).astype('float32')
np.save("zara_features_efficientnet.npy", zara_features_efficientnet)

# Cleanup
del efficientnet_model
tf.keras.backend.clear_session()
import gc
gc.collect()

I0000 00:00:1750619125.470354      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15451 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


  0%|          | 0/235 [00:00<?, ?it/s]

I0000 00:00:1750619149.569904     110 service.cc:148] XLA service 0x7f6b20004350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750619149.570667     110 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750619152.407831     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750619163.238884     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  0%|          | 0/27 [00:00<?, ?it/s]

0

**RESTART KERNEL HERE before moving to next model** 

(Use the restart button at the top of your notebook, then rerun only the "Preparation" cell and the next model’s cell.)

## 2️⃣ ConvNeXtBase Feature Extraction

In [15]:
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.applications.convnext import preprocess_input as convnext_preprocess

convnext_model = ConvNeXtBase(weights="imagenet", include_top=False, pooling="avg")

def extract_features_batch_convnext(img_paths):
    batch_images = []
    for img_path in img_paths:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        batch_images.append(x)
    batch_images = np.array(batch_images)
    batch_images = convnext_preprocess(batch_images)
    feats = convnext_model.predict(batch_images, batch_size=len(batch_images), verbose=0)
    return feats

features_convnext = []
for start in tqdm(range(0, len(img_paths), BATCH_SIZE)):
    batch = img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_convnext(batch)
    features_convnext.extend(feats)
features_convnext = np.array(features_convnext).astype('float32')
np.save("features_convnext.npy", features_convnext)

zara_features_convnext = []
for start in tqdm(range(0, len(zara_img_paths), BATCH_SIZE)):
    batch = zara_img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_convnext(batch)
    zara_features_convnext.extend(feats)
zara_features_convnext = np.array(zara_features_convnext).astype('float32')
np.save("zara_features_convnext.npy", zara_features_convnext)

# Cleanup
del convnext_model
tf.keras.backend.clear_session()
gc.collect()

350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

0

**RESTART KERNEL HERE before moving to next model**

## 3️⃣ ViT (Hugging Face) Feature Extraction

In [7]:
import torch
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
vit_model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
vit_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

def extract_features_batch_vit(img_paths):
    images = [Image.open(p).convert("RGB").resize((224, 224)) for p in img_paths]
    inputs = vit_processor(images=images, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = vit_model(**inputs)
        feats = outputs.pooler_output
        feats = feats / feats.norm(dim=1, keepdim=True)
    return feats.cpu().numpy()

# Set a SMALL batch size to avoid OOM errors!
BATCH_SIZE = 32

# ---- For main dataset ----
features_vit = []
for start in tqdm(range(0, len(img_paths), BATCH_SIZE)):
    batch = img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_vit(batch)
    features_vit.extend(feats)
    # Free up GPU RAM after every batch (for extra safety)
    torch.cuda.empty_cache()
    gc.collect()
features_vit = np.array(features_vit).astype('float32')
np.save("features_vit.npy", features_vit)

# ---- For Zara queries ----
zara_features_vit = []
for start in tqdm(range(0, len(zara_img_paths), BATCH_SIZE)):
    batch = zara_img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_vit(batch)
    zara_features_vit.extend(feats)
    torch.cuda.empty_cache()
    gc.collect()
zara_features_vit = np.array(zara_features_vit).astype('float32')
np.save("zara_features_vit.npy", zara_features_vit)

# Cleanup: clear ViT model from memory
del vit_model
torch.cuda.empty_cache()
gc.collect()

  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

17

**RESTART KERNEL HERE before moving to next model**

## 4️⃣ CLIP (Hugging Face) Feature Extraction

In [10]:
import numpy as np
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from tqdm.notebook import tqdm
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
BATCH_SIZE = 32  # For CLIP, keep batch size small

def extract_features_batch_clip(img_paths):
    images = [Image.open(p).convert("RGB").resize((224, 224)) for p in img_paths]
    inputs = clip_processor(images=images, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        image_features = clip_model.get_image_features(**inputs)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
    return image_features.cpu().numpy()

# Main dataset
features_clip = []
for start in tqdm(range(0, len(img_paths), BATCH_SIZE)):
    batch = img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_clip(batch)
    features_clip.extend(feats)
    torch.cuda.empty_cache()
    gc.collect()
features_clip = np.array(features_clip).astype('float32')
np.save("features_clip.npy", features_clip)

# Zara queries
zara_features_clip = []
for start in tqdm(range(0, len(zara_img_paths), BATCH_SIZE)):
    batch = zara_img_paths[start:start+BATCH_SIZE]
    feats = extract_features_batch_clip(batch)
    zara_features_clip.extend(feats)
    torch.cuda.empty_cache()
    gc.collect()
zara_features_clip = np.array(zara_features_clip).astype('float32')
np.save("zara_features_clip.npy", zara_features_clip)

# Clean up
del clip_model
torch.cuda.empty_cache()
gc.collect()


  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

17

**After all models are done, you can load the .npy files for FAISS search/visualization. No restart needed after this.**

This approach guarantees you won’t run out of GPU memory. Just remember:
“Restart kernel, run the data prep, run ONE model, save features, cleanup, and move to the next.”